In [1]:
import os
# print(os.getcwd())
os.chdir('C:/Users/eddie/Desktop/Prof. Rumi Chunara/alcohol')
import pickle
import sklearn
print(sklearn.__version__)
import pandas
import numpy as np
import sys


from evaluation.metric import computeAccuracy, computeF1Score, computeAUC
# from sklearn.metrics import roc_auc_score

# %load_ext autoreload
# %autoreload
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from scipy.stats import uniform
# %reload_ext pipelines.alcohol.AlcoholPipeline
from pipelines.alcohol import AlcoholPipeline

0.17


In [2]:
# clf_alc_initial = pickle.load(open('./alcohol classifiers/clf_alc_UPDATED.p', 'rb'))
clf_fpa_initial = pickle.load(open('./alcohol classifiers/clf_fpa_UPDATED.p', 'rb'))
# clf_fpl_dl = pickle.load(open('./alcohol classifiers/clf_fpl_double_labeled', 'rb'))
df = pandas.read_csv("./data/alcohol_training_instances.csv")

Prepare the first person level data

In [3]:
fp_labels = []
fp_index_in_df = []
# index = 

fp_0 = "'first_person': 0"
fp_1 = "'first_person': 1"

# the actual extraction
for i in range(len(df)):
    if fp_0 in df.labels[i] or fp_1 in df.labels[i]:
        fp_index_in_df.append(1)
        if df.labels[i].find(fp_0) != -1:
            fp_labels.append(df.labels[i][df.labels[i].find(fp_0) + len(fp_0) - 1])
        else:
            fp_labels.append(df.labels[i][df.labels[i].find(fp_1) + len(fp_0) - 1])
    else:
        fp_index_in_df.append(0)
fp_labels = list(map(int, fp_labels))
print(fp_labels[:20])
# print(subset_vec_alc_fp[:10])
print('length of first person level labels: ', len(fp_labels)) # 9816
print(fp_index_in_df[:20])
print('length of first person level index in df: ', len(fp_index_in_df)) # 15651
index_fp = list(np.where(fp_index_in_df)[0])
print(index_fp[:15])
print('length of first person level df index: ', len(index_fp)) # 9816
df_fp = df.iloc[index_fp,]
# df_fp.head
# fp_labels = np.asarray(fp_labels)

# train test dataset split
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_fp, fp_labels, test_size=0.2, random_state=26)

[0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1]
length of first person level labels:  9816
[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
length of first person level index in df:  15651
[1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 17]
length of first person level df index:  9816


First we check the initial model performance

In [4]:
%%time
clf_fpa_initial.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Wall time: 5min 41s


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=F...probability=True,
  random_state=None, shrinking=True, tol=0.0008753898561476732,
  verbose=False))])

In [5]:
# print(type(y_test))
# print(y_test[:10])
# # computeAccuracy(y_fpa_initial, y_test)
# auc_score_initial = computeAUC(clf_fpa_initial, X_test, y_test, plot=False)

In [6]:
# predict in testing set.
y_fpa_initial = clf_fpa_initial.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_fpa_initial, y_test)
f1_initial = computeF1Score(y_test, y_fpa_initial)
print('F1 Score: ', f1_initial)
auc_score_initial = computeAUC(clf_fpa_initial, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_initial)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1964
pretrained alcohol classifer:  626 different item in whole dataset
accuracy: 68.126273%
F1 Score:  0.7904953145917002
alcohol LR AUC:  0.7374843994644761


Then we make our own SVM model on first person level

In [7]:
# SVM
clf_fp_SVM = AlcoholPipeline(global_features=["text"]).pipeline(LinearSVC())
clf_fp_SVM.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=None, min_df=1,
      ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0))]

In [8]:
clf_fp_SVM.fit(X_train, y_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Pipeline(steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=T...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

Predict in testing set

In [10]:
# predict in testing set.
y_fp_SVM = clf_fp_SVM.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_fp_SVM, y_test)
f1_SVM = computeF1Score(y_test, y_fp_SVM)
print('F1 Score: ', f1_SVM)
auc_score_SVM = computeAUC(clf_fp_SVM, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_SVM)

length of total comps:  1964
pretrained alcohol classifer:  646 different item in whole dataset
accuracy: 67.107943%
F1 Score:  0.7770876466528641
alcohol LR AUC:  0.6733644965849009


D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


Test in training set to figure out why the test accuracy is low

In [11]:
# test in training set to figure out why the test accuracy is low
y_train_fp_SVM = clf_fp_SVM.predict(X_train)
computeAccuracy(y_train_fp_SVM, y_train)
f1_score_SVM_train = computeF1Score(y_train, y_train_fp_SVM)
print('F1 Score: ', f1_score_SVM_train)
auc_score_SVM_train = computeAUC(clf_fp_SVM, X_train, y_train, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_SVM_train)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  7852
pretrained alcohol classifer:  2495 different item in whole dataset
accuracy: 68.224656%
F1 Score:  0.7848952495904821
alcohol LR AUC:  0.7143992185164396


We can see the performance in training set is also not good. So there must be sth wrong within the model structure parameters, since the model structure are same. Please see compareStructure.ipynb for visualization.

Update structure parameters in SVM model. To make sure they become same with the initial model

In [12]:
clf_fp_SVM_updateParams = AlcoholPipeline(global_features=["text"]).pipeline(LinearSVC())
params_SVM = {'features__text__tfidf__dtype':np.int64, 'features__text__tfidf__max_features':86124, 'features__text__tfidf__ngram_range':(1,3) 
            }
# to see all parameters in model
# clf_alc_LR_updateParams.get_params().keys()
clf_fp_SVM_updateParams.set_params(**params_SVM)
clf_fp_SVM_updateParams.steps

[('features', FeatureUnion(n_jobs=1,
         transformer_list=[('text', Pipeline(steps=[('getter', ItemGetter(key='text')), ('tfidf', TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
          dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
          lowercase=True, max_df=1.0, max_features=86124, min_df=1,
     ...      token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
          vocabulary=None))]))],
         transformer_weights=None)),
 ('scaler', Normalizer(copy=True, norm='l2')),
 ('clf', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
       intercept_scaling=1, loss='squared_hinge', max_iter=1000,
       multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
       verbose=0))]

Train the SVM model with updated parameters, and predict in testing set

In [13]:
clf_fp_SVM_updateParams.fit(X_train, y_train)
# predict in testing set.
y_fp_SVM_updateParams = clf_fp_SVM_updateParams.predict(X_test)
# print('length of testing set: ', len(y_alc_initial))
computeAccuracy(y_fp_SVM_updateParams, y_test)
f1_score_SVM_updateParams = computeF1Score(y_test, y_fp_SVM_updateParams)
print('F1 Score: ', f1_score_SVM_updateParams)
auc_score_SVM_updateParams = computeAUC(clf_fp_SVM_updateParams, X_test, y_test, plot=False)
# auc_score = computeAUC(clf_alc_LR_updateParams, X_train, y_train, X_test, y_test, plot=True, plotTitle="Alcohol ROC Curve")
print('alcohol LR AUC: ', auc_score_SVM_updateParams)

D:\Anaconda3\envs\py36\lib\site-packages\sklearn\feature_extraction\text.py:1015: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


length of total comps:  1964
pretrained alcohol classifer:  584 different item in whole dataset
accuracy: 70.264766%
F1 Score:  0.7804511278195488
alcohol LR AUC:  0.7262418026276974
